In [1]:
%pip install tensorflow==2.5.0 scikit-learn optuna

ERROR: Could not find a version that satisfies the requirement tensorflow==2.5.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.5.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [3]:
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

In [4]:
import tensorflow as tf

2025-02-21 16:48:07.576404: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-21 16:48:07.579830: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-21 16:48:07.588014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740136687.601781    5182 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740136687.605493    5182 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 16:48:07.620316: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [5]:
print(tf.__version__)
print(np.__version__)

2.18.0
2.0.2


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [7]:
def get_mlp_model(input_shape,hidden_layer_one,dropout_one,hidden_layer_two,dropout_two):
    model = Sequential()
    model.add(Dense(hidden_layer_one, activation='elu', input_shape=input_shape))
    if dropout_one !=0:
        model.add(Dropout(dropout_one))
    model.add(Dense(hidden_layer_two, activation='elu'))
    if dropout_two !=0:
        model.add(Dropout(dropout_two))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer="Adam")

    return model

In [8]:
def compute_error(trues, predicted):
    corr = np.corrcoef(predicted, trues)[0,1]
    mae = np.mean(np.abs(predicted - trues))
    rae = np.sum(np.abs(predicted - trues)) / np.sum(np.abs(trues - np.mean(trues)))
    rmse = np.sqrt(np.mean((predicted - trues)**2))
    rrse = np.sqrt(np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    mape = np.mean(np.abs((predicted - trues) / trues)) * 100
    r2 = max(0, 1 - np.sum((predicted - trues)**2) / np.sum((trues - np.mean(trues))**2))
    # Calculez les autres mesures d'erreur ici
    return rmse, corr, mae, rae, rrse, mape, r2

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import optuna
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.neural_network import MLPRegressor
from tensorflow.keras.callbacks import ModelCheckpoint

/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

W0000 00:00:1740136689.900728    5182 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

# We start by loading the pre-processed data

In [11]:
order_history_data=pd.read_csv('../data/final_data.csv')
order_history_data.head()

,Inflation,Month,Year,pmi,PartNo,Description,Quantity,Sales Price,Total Sales,Swaraj Engines Stock Price,M&M Stock Price,Escorts Kuboto Stock Price
0,7.59,1,2020,250,0503BA0290N-RK,ALFA HUB,85,2118.60,59367,1125.0,532.9,630.0
1,7.59,1,2020,250,0503BA0290N,Front Wheel Hub,318,3536.35,59367,1125.0,532.9,630.0
2,7.59,1,2020,250,0502BA1050N,Alfa Rear Wheel Hub LH,209,1383.36,59367,1125.0,532.9,630.0
3,7.59,1,2020,250,0502BA1060N,Alfa Rear Wheel Hub RH,119,4849.50,59367,1125.0,532.9,630.0
4,7.59,1,2020,250,0602AAB00260N,Brake Drum ALFA,134,2462.26,59367,1125.0,532.9,630.0


# Deep learning model optimization

In [12]:
X = order_history_data.copy()
y = order_history_data['Quantity']

X_encoded = pd.get_dummies(X, columns=['Description', 'PartNo'])

X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y, test_size=0.3, random_state=27)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=27)

We're going to use mlp. To start with, we'll create an mlp with minimal parameters to get an idea of performance without model optimization. Then we'll optmize our model using Bayesian optimization.

In [13]:

best_mlp = MLPRegressor(hidden_layer_sizes=(50), activation='relu', alpha=0.008724528119026307, learning_rate='constant')

# Train the model on the training data
best_mlp.fit(X_train, y_train)

# Make predictions on the validation set with the trained model
y_val_pred = best_mlp.predict(X_val)

# Evaluate the performance of the model on the validation set
mse_val = mean_squared_error(y_val, y_val_pred)
r2_val = best_mlp.score(X_val, y_val)

# Make predictions on the test set with the trained model
y_test_pred = best_mlp.predict(X_test)

# Evaluate the performance of the model on the test set
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = best_mlp.score(X_test, y_test)

print("Validation Set:")
print("MSE (Mean Squared Error) on the validation set:", mse_val)
print("R-squared (Coefficient of Determination) on the validation set:", r2_val)

print("\nTest Set:")
print("MSE (Mean Squared Error) on the test set:", mse_test)
print("R-squared (Coefficient of Determination) on the test set:", r2_test)



Validation Set:
MSE (Mean Squared Error) on the validation set: 8.52469529733593
R-squared (Coefficient of Determination) on the validation set: 0.9996039578120539

Test Set:
MSE (Mean Squared Error) on the test set: 9.412318902541893
R-squared (Coefficient of Determination) on the test set: 0.9995810905170607


Using Bayesian optimization to find the best hyperparameters

In [14]:
# Objective function for Optuna optimization
def objective(trial):
    # Define the hyperparameters to be optimized
    hidden_layer_one = trial.suggest_categorical('hidden_layer_one', [32, 64, 128, 256,512])
    hidden_layer_two = trial.suggest_categorical('hidden_layer_two', [32, 64, 128, 256,512])
    dropout_one = trial.suggest_uniform('dropout_one', 0, 0.35)
    dropout_two = trial.suggest_uniform('dropout_two', 0, 0.5)
    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128,256,512])
    epochs = trial.suggest_categorical('epochs', [150])



    model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)


    # Define the ModelCheckpoint callback
    checkpoint_filepath = 'model_checkpoint.h5'
    model_checkpoint = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_best_only=True,
        monitor='val_loss',
        mode='min',
        verbose=0
    )

    # Train the model on the training data with validation data and checkpoint callback
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0
                        , validation_data=(X_val, y_val),
                        callbacks=[model_checkpoint])

    # Load the best weights from the saved checkpoint
    model.load_weights(checkpoint_filepath)

    # Evaluate the performance of the model on the test set
    y_test_pred = model.predict(X_test)

    # Calculate performance metrics
    mse_test = mean_squared_error(y_test, y_test_pred)
    # rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

    return mse_test

# Configure Optuna to use the GPU for exhaustive searches
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100, n_jobs=1)  # Use n_jobs=1 to avoid parallelism problems on GPU

# Show best hyperparameters found
print("Best hy    perparameters:")
print(study.best_params)
print("Best MSE:", study.best_value)

In [17]:
best_params = study.best_params
best_mse = study.best_value

NameError: name 'study' is not defined

Using Grid Search

In [15]:
# r2=0

# for hidden_layer_one in [25,50,75,100,125,150,175,200,225,350,300,350,400]:
#   for hidden_layer_two in [325,50,75,100,125,150,175,200,225,350,300,350,400]:
#       for dropout_one in [0,0.1,0.2,0.3,0.4,0.5]:
#         for dropout_two in [0,0.1,0.2,0.3,0.4,0.5]:
#           # for batch_size in [32,64,128,256]:


#             model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

#             # Define the ModelCheckpoint callback
#             checkpoint_filepath = 'model_checkpoint.h5'
#             model_checkpoint = ModelCheckpoint(
#                 filepath=checkpoint_filepath,
#                 save_best_only=True,
#                 monitor='val_loss',
#                 mode='min',
#                 verbose=0
#             )

#             # Train the model on the training data with validation data and checkpoint callback
#             history = model.fit(X_train, y_train, epochs=300, batch_size=256,
#                                 verbose=0, validation_data=(X_val, y_val),
#                                 callbacks=[model_checkpoint])

#             # Load the best weights from the saved checkpoint
#             model.load_weights(checkpoint_filepath)

#             # Evaluate the performance of the model on the test set
#             y_test_pred = model.predict(X_test)

#             # Calculate performance metrics
#             mse_test = mean_squared_error(y_test, y_test_pred)
#             rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

#             if r2_2>r2:
#               r2=r2_2
#               print("{},{},{},{}".format(hidden_layer_one,hidden_layer_two,dropout_one,dropout_two) )
#               print("RMSE:", rmse2)
#               print("Corrélation:", corr2)
#               print("MAE:", mae2)
#               print("RAE:", rae2)
#               print("RRSE:", rrse2)
#               print("MAPE:", mape2)
#               print("R2:", r2_2)
#               print("MSE:", mse_test)
#               print("----------------------------------")

/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 

Training model with best hyperparamters

In [16]:
# best parameters

hidden_layer_one=32
hidden_layer_two=64
dropout_one=0.3
dropout_two=0.15


model=get_mlp_model(input_shape=(X_train.shape[1],),hidden_layer_one=hidden_layer_one,dropout_one=dropout_one,hidden_layer_two=hidden_layer_two,dropout_two=dropout_two)

# Define the ModelCheckpoint callback
checkpoint_filepath = '../checkpoints/bayesian_checkpoint.h5'
model_checkpoint = ModelCheckpoint(
filepath=checkpoint_filepath,
save_best_only=True,
monitor='val_loss',
mode='min',
verbose=0
)

# Train the model on the training data with validation data and checkpoint callback
history = model.fit(X_train, y_train, epochs=300, batch_size=512,
            verbose=2, validation_data=(X_val, y_val),
            callbacks=[model_checkpoint])

# Load the best weights from the saved checkpoint
model.load_weights(checkpoint_filepath)

# Evaluate the performance of the model on the test set
y_test_pred = model.predict(X_test)

# Calculate performance metrics
rmse2, corr2, mae2, rae2, rrse2, mape2, r2_2 = compute_error(y_test.values, y_test_pred.reshape(y_test_pred.shape[0]))

print("RMSE:", rmse2)
print("Corrélation:", corr2)
print("MAE:", mae2)
print("RAE:", rae2)
print("RRSE:", rrse2)
print("MAPE:", mape2)
print("R2:", r2_2)
print("----------------------------------")

Epoch 1/300


/home/usman/miniconda3/envs/curie/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 - 1s - 49ms/step - loss: 4363630.0000 - val_loss: 239383.0156
Epoch 2/300


15/15 - 0s - 6ms/step - loss: 1634522.7500 - val_loss: 71554.6797
Epoch 3/300


15/15 - 0s - 6ms/step - loss: 913646.9375 - val_loss: 29128.1953
Epoch 4/300


...

15/15 - 0s - 5ms/step - loss: 9854.1172 - val_loss: 4226.7612
Epoch 51/300


15/15 - 0s - 6ms/step - loss: 9735.3447 - val_loss: 4132.3711
Epoch 52/300
15/15 - 0s - 5ms/step - loss: 9030.2920 - val_loss: 7067.3594
Epoch 53/300
15/15 - 0s - 5ms/step - loss: 10001.8203 - val_loss: 4739.8496
Epoch 54/300


15/15 - 0s - 6ms/step - loss: 9507.7959 - val_loss: 1744.2216
Epoch 55/300
15/15 - 0s - 5ms/step - loss: 9431.8564 - val_loss: 6642.9541
Epoch 56/300
15/15 - 0s - 5ms/step - loss: 9646.8564 - val_loss: 4147.3345
Epoch 57/300
15/15 - 0s - 5ms/step - loss: 8735.5596 - val_loss: 3534.4478
Epoch 58/300
15/15 - 0s - 5ms/step - loss: 8553.0283 - val_loss: 2204.8823
Epoch 59/300
15/15 - 0s - 5ms/step - loss: 8608.4678 - val_loss: 2213.6519
Epoch 60/300


15/15 - 0s - 6ms/step - loss: 8236.3213 - val_loss: 1386.9896
Epoch 61/300
15/15 - 0s - 7ms/step - loss: 8251.7598 - val_loss: 2963.3032
Epoch 62/300
15/15 - 0s - 5ms/step - loss: 8115.0029 - val_loss: 2711.2764
Epoch 63/300
15/15 - 0s - 5ms/step - loss: 7744.2705 - val_loss: 3036.2964
Epoch 64/300
15/15 - 0s - 5ms/step - loss: 7717.1382 - val_loss: 2236.1084
Epoch 65/300
15/15 - 0s - 5ms/step - loss: 7457.0566 - val_loss: 5274.8145
Epoch 66/300
15/15 - 0s - 5ms/step - loss: 7679.0366 - val_loss: 2786.0229
Epoch 67/300
15/15 - 0s - 5ms/step - loss: 8216.3887 - val_loss: 3975.7783
Epoch 68/300
15/15 - 0s - 5ms/step - loss: 7343.6768 - val_loss: 5783.5122
Epoch 69/300
15/15 - 0s - 5ms/step - loss: 7122.1890 - val_loss: 5384.1919
Epoch 70/300
15/15 - 0s - 5ms/step - loss: 7227.8843 - val_loss: 3261.4292
Epoch 71/300
15/15 - 0s - 5ms/step - loss: 6937.3628 - val_loss: 4708.6987
Epoch 72/300
15/15 - 0s - 5ms/step - loss: 6878.0171 - val_loss: 4282.5093
Epoch 73/300
15/15 - 0s - 5ms/step - l

15/15 - 0s - 6ms/step - loss: 5411.8369 - val_loss: 1118.9254
Epoch 86/300
15/15 - 0s - 6ms/step - loss: 5724.7212 - val_loss: 4482.6333
Epoch 87/300
15/15 - 0s - 5ms/step - loss: 5078.3965 - val_loss: 5438.6953
Epoch 88/300
15/15 - 0s - 5ms/step - loss: 5084.4717 - val_loss: 5483.6504
Epoch 89/300
15/15 - 0s - 5ms/step - loss: 5110.7324 - val_loss: 3518.7659
Epoch 90/300
15/15 - 0s - 5ms/step - loss: 5085.8057 - val_loss: 2869.1829
Epoch 91/300
15/15 - 0s - 5ms/step - loss: 5247.4863 - val_loss: 6764.8081
Epoch 92/300
15/15 - 0s - 5ms/step - loss: 4833.8994 - val_loss: 6873.5400
Epoch 93/300
15/15 - 0s - 5ms/step - loss: 4657.0068 - val_loss: 4220.2407
Epoch 94/300
15/15 - 0s - 5ms/step - loss: 4361.8750 - val_loss: 3840.3740
Epoch 95/300
15/15 - 0s - 5ms/step - loss: 4646.6021 - val_loss: 3648.5317
Epoch 96/300
15/15 - 0s - 5ms/step - loss: 4397.7192 - val_loss: 6092.9995
Epoch 97/300
15/15 - 0s - 5ms/step - loss: 4318.3872 - val_loss: 3866.4941
Epoch 98/300
15/15 - 0s - 5ms/step - l

Save the weights of the best model for later loading

In [19]:
# Save the best model in checkpoint only if it is better
model.save_weights('../checkpoint/grid_search_checkpoint.weights.h5')